In [ ]:
import requests
from selenium import webdriver
import time
import datetime
from pyquery import PyQuery as pq

In [ ]:
# 執行webdirver、輸入網址
driver = webdriver.Chrome("chromedriver")
youtuberUrl = input('請輸入Youtuber頻道網址')
driver.get("{}".format(youtuberUrl))

In [ ]:
# Youtuber簡介、加入日期、總觀看次數
driver.find_element_by_css_selector("#tabsContent > paper-tab:nth-child(12)").click()
time.sleep(1)
intro = driver.find_element_by_css_selector("*").get_attribute("outerHTML")
doc = pq(intro)
begindate = doc("#right-column > yt-formatted-string:nth-child(2)").text()
print(begindate)
allviews = doc("#right-column > yt-formatted-string:nth-child(3)").text()
print(allviews)

In [ ]:
# 頻道經營天數、現在是第幾季
datetime.datetime.now()
beginDate = begindate.strip("加入日期：日").replace("月","-").replace("年","-")
BeginDate = datetime.datetime.strptime(beginDate,'%Y-%m-%d')
nowdate = datetime.datetime.now()
begindays = (nowdate - BeginDate).days
print("此頻道經營了 "+str(begindays)+" 天")

# 這一季
# 一季預設 90 天
seasons = int(begindays) / 90
print("現在是第 " + str(int(seasons)) + " 季")

In [ ]:
# 頁面捲動
driver.find_element_by_css_selector("#tabsContent > paper-tab:nth-child(4)").click()
time.sleep(1)
for uu in range(10):
    js = "var action = document.documentElement.scrollTop=10000"
    driver.execute_script(js)
    time.sleep(3)

# 頁面高度
element = driver.find_element_by_css_selector("body > ytd-app")
h = element.size["height"]
h

# 擷取網頁資料
html = driver.find_element_by_css_selector("*").get_attribute("outerHTML")
doc = pq(html)

In [ ]:
# 訂閱人數
subs = doc("#subscriber-count").text()
print(subs)
if "萬" in subs:
    subS = float(subs.replace("萬 位訂閱者",""))*10000
else:
    subS = int(subs)

if subS > 1000000:
    print("大型Youtuber")
elif subS > 100000:
    print("中型Youtuber")
elif subS > 10000:
    print("小型Youtuber")
else:
    print("微型Youtuber")


In [ ]:
# 這一季
allViews = allviews.replace("觀看次數：","").replace(",",'')
print("第 " + str(int(seasons)) + " 季")

# 標題、觀看次數、發佈時間、影片長度資料放置處
dataList = []
for eachitemDoc in doc("#contents #dismissable").items():
    dataDict = {}
    dataDict["title"] = eachitemDoc("#video-title").text()
    dataDict["views"] = eachitemDoc("#metadata-line > span:nth-child(1)").text()
    dataDict["ago"] = eachitemDoc("#metadata-line > span:nth-child(2)").text()
    if dataDict["ago"] == '4 個月前':
        break
    dataDict["time"] = eachitemDoc("#overlays > ytd-thumbnail-overlay-time-status-renderer > span").text()
    dataList.append(dataDict)

# 有幾部影片
print("總共 "+str(len(dataList))+" 部影片")

# 標題
totaltitle = ""
for i in range(0, len(dataList)):
    eachtitle = dataList[i]
    totaltitle += eachtitle["title"]

# 影片標題總共有幾個字
print("標題總字數 : "+str(len(totaltitle))+" 字")

# 每部影片標題平均幾個字
averageTitle = float(len(totaltitle))/int(len(dataList))
print("標題平均字數 : "+str(round(averageTitle,0))+" 字")


# 觀看次數
totalviews = 0
for j in range(0, len(dataList)):
    eachviews = dataList[j]
    eachv = eachviews["views"].strip("觀看次數：次")
    if "萬" in eachv:
        eachv = float(eachv.replace("萬",""))*10000
    elif "億" in eachv:
        eachv = float(eachv.replace("億",""))*100000000
    else:
        eachv = int(eachv)*1
    totalviews += eachv

# 總觀看次數
print("總觀看次數 : "+str(int(totalviews))+" 次")

# 平均觀看次數
averageviews = round(int(totalviews) / int(len(dataList)),0)
print("平均觀看次數 : "+str(averageviews)+" 次")

# 營利播放次數 (預設係數為0.6)
totalRviews = totalviews*0.6
print("營利播放次數 : "+str(round(totalRviews,0))+" 次")

# Youtube 廣告收益
# CPM(每千次廣告播放曝光次數的預估平均總收益)在台灣大約1.5美金
revenueYoutube = 1.5*totalRviews/1000
print("Youtube廣告收益 : "+str(round(revenueYoutube,0))+" 美金")

# 頻道主廣告收益 (預設係數為0.55)
revenueYoutuber = round(0.55*revenueYoutube,0)
print("頻道主廣告收益 : "+str(revenueYoutuber)+" 美金")

# 影片時數
totalseconds = 0
for k in range(0, len(dataList)):
    eachseconds = dataList[k]["time"].split(":")
    if len(eachseconds) == 3:
        totalseconds += int(eachseconds[0])*3600 + int(eachseconds[1])*60 + int(eachseconds[2])
    else:
        totalseconds += int(eachseconds[0])*60 + int(eachseconds[1])
        
totalminutes = int(totalseconds / 60)
averagetime = round(totalminutes / len(dataList), 0)

# 影片總長度 
print("影片總長度 : "+str(totalminutes)+" 分鐘")

# 影片平均長度
print("平均影片長度 : "+str(averagetime)+" 分鐘")

# 這一季總觀看次數 / 總觀看次數
aviewspercentage = int(totalviews) / int(allViews)
print("第 " + str(int(seasons)) + " 季觀看次數佔頻道總觀看次數的 {:.2%}".format(aviewspercentage))


In [ ]:
# 觀眾黏著度
# 公式 : 平均觀看次數 / 訂閱數
youtuberPopularity = averageviews / subS
print("觀眾黏著度 : {:.2%}".format(youtuberPopularity))

if float(youtuberPopularity) < 0.5:
    print("再加油")
elif float(youtuberPopularity) < 0.8:
    print("尚可")
elif float(youtuberPopularity) < 1:
    print("不錯")
else:
    print("超棒")
    


In [ ]:
# 上一季
allViews = allviews.replace("觀看次數：","").replace(",",'')
seasons = int(begindays) / 90
Lseasons = int(seasons)-1
print("第 " + str(Lseasons) + " 季")

# 上一季標題、觀看次數、發布時間、影片長度資料放置處
LSdataList = []
for LSeachitemDoc in doc("#contents #dismissable").items():
    LSdataDict = {}
    LSdataDict["ago"] = LSeachitemDoc("#metadata-line > span:nth-child(2)").text()
    if "分" in LSdataDict["ago"]:
        continue
    elif "時" in LSdataDict["ago"]:
        continue
    elif "天" in LSdataDict["ago"]:
        continue
    elif "週" in LSdataDict["ago"]:
        continue
    elif "1 個月前" in LSdataDict["ago"]:
        continue
    elif "2 個月前" in LSdataDict["ago"]:
        continue
    elif "3 個月前" in LSdataDict["ago"]:
        continue
    elif LSdataDict["ago"] == '7 個月前':
        break
    LSdataDict["title"] = LSeachitemDoc("#video-title").text()
    LSdataDict["views"] = LSeachitemDoc("#metadata-line > span:nth-child(1)").text()
    LSdataDict["time"] = LSeachitemDoc("#overlays > ytd-thumbnail-overlay-time-status-renderer > span").text()
    LSdataList.append(LSdataDict)
    
# 有幾部影片
print("總共 "+str(len(LSdataList))+" 部影片")

# 標題
LStotaltitle = ""
for ii in range(0, len(LSdataList)):
    LSeachtitle = LSdataList[ii]
    LStotaltitle += LSeachtitle["title"]

# 影片標題總共有幾個字
print("標題總字數 : "+str(len(LStotaltitle)) + " 字")

# 平均每部影片標題幾個字
LSaverageTitle = float(len(LStotaltitle))/int(len(LSdataList))
print("標題平均字數 : "+str(round(LSaverageTitle,0)) + " 字")

# 觀看次數
LStotalviews = 0
for jj in range(0, len(LSdataList)):
    LSeachviews = LSdataList[jj]
    LSeachv = LSeachviews["views"].replace("觀看次數：","").replace("次","")
    if "萬" in LSeachv:
        LSeachv = float(LSeachv.replace("萬",""))*10000
    elif "億" in LSeachv:
        LSeachv = float(LSeachv.replace("億",""))*100000000
    else:
        LSeachv = int(LSeachv)*1
    LStotalviews += LSeachv
    
# 總觀看次數
print("總觀看次數 : "+str(int(LStotalviews))+" 次")

# 平均觀看次數
LSaverageviews = round(int(LStotalviews)/int(len(LSdataList)),0)
print("平均觀看次數 : "+str(LSaverageviews)+" 次")

# 營利播放次數 (預設係數為0.6)
LStotalRviews = LStotalviews*0.6
print("營利播放次數 : "+str(round(LStotalRviews,0))+" 次")

# Youtube 廣告收益
# CPM(每千次廣告播放曝光次數的預估平均總收益)在台灣大約1.5美金
LSrevenueYoutube = 1.5*LStotalRviews/1000
print("Youtube廣告收益 : "+str(round(LSrevenueYoutube,0))+" 美金")

# 頻道主廣告收益 (預設係數為0.55)
LSrevenueYoutuber = round(0.55*LSrevenueYoutube,0)
print("頻道主廣告收益 : "+str(LSrevenueYoutuber)+" 美金")

# 影片時數
LStotalseconds = 0
for kk in range(0, len(LSdataList)):
    LSeachseconds = LSdataList[kk]["time"].split(":")
    if len(LSeachseconds) == 3:
        LStotalseconds += int(LSeachseconds[0])*3600 + int(LSeachseconds[1])*60 + int(LSeachseconds[2])
    else:
        LStotalseconds += int(LSeachseconds[0])*60 + int(LSeachseconds[1])

LStotalminutes = int(LStotalseconds / 60)
LSaveragetime = round(LStotalminutes / len(LSdataList), 0)

# 影片總長度 
print("影片總長度 : "+str(LStotalminutes)+" 分鐘")

# 影片平均長度
print("平均影片長度 : "+str(LSaveragetime)+" 分鐘")

# 這一季總觀看次數 / 總觀看次數
aviewspercentage = int(LStotalviews) / int(allViews)
print("第 " + str(Lseasons) + " 季觀看次數佔頻道總觀看次數的 {:.2%}".format(aviewspercentage))


In [ ]:
# 比較兩季表現

# 影片總數
dataListdif = len(dataList) - len(LSdataList)
if dataListdif < 0:
    print("這一季影片總數較上一季減少 {} 部".format(abs(dataListdif)))
else:
    print("這一季影片總數較上一季增加 {} 部".format(abs(dataListdif)))

# 總觀看次數
totalviewsdif = int(totalviews) - int(LStotalviews)
totalviewsDif = totalviewsdif / int(totalviews)
if totalviewsDif < 0:
    print("這一季總觀看次數較上一季減少 {:.2%}".format(abs(totalviewsDif)))
else:
    print("這一季總觀看次數較上一季增加 {:.2%}".format(abs(totalviewsDif)))
    
# 平均觀看次數
averageviewsdif = averageviews - LSaverageviews
averageviewsDif = averageviewsdif / averageviews
if averageviewsDif < 0:
    print("這一季平均觀看次數較上一季減少 {:.2%}".format(abs(averageviewsDif)))
else:
    print("這一季平均觀看次數較上一季增加 {:.2%}".format(abs(averageviewsDif)))
    
# 頻道主廣告收益
revenueYoutuberdif = revenueYoutuber - LSrevenueYoutuber
revenueYoutuberDif = revenueYoutuberdif / revenueYoutuber
if revenueYoutuberDif < 0:
    print("這一季頻道主廣告收益較上一季減少 {:.2%}".format(abs(revenueYoutuberDif)))
else:
    print("這一季頻道主廣告收益較上一季增加 {:.2%}".format(abs(revenueYoutuberDif)))

# 影片總長度
totalminutesdif = totalminutes - LStotalminutes
totalminutesDif = totalminutesdif / totalminutes
if totalminutesDif < 0:
    print("這一季影片總長度較上一季減少 {:.2%}".format(abs(totalminutesDif)))
else:
    print("這一季影片總長度較上一季增加 {:.2%}".format(abs(totalminutesDif)))
    
# 平均影片長度
averagetimedif = averagetime - LSaveragetime
averagetimeDif = averagetimedif / averagetime
if averagetimeDif < 0:
    print("這一季平均影片長度較上一季減少 {:.2%}".format(abs(averagetimeDif)))
else:
    print("這一季平均影片長度較上一季增加 {:.2%}".format(abs(averagetimeDif)))
    

In [ ]:
driver.quit()